In [2]:

import pandas as pd
import main
las_gis_file_name = "fake_data\\fake_gis_las_gr_out.csv"
las_data = pd.read_csv(las_gis_file_name, delimiter=';')
las_data['WELL_NAME'] = las_data['WELL_NAME'].astype('str')
las_data['WELL_NAME_UWI'] = las_data['WELL_NAME_UWI'].astype('str')
las_data.info()

D:\libs\Anaconda\3.3\envs\aind\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5701762 entries, 0 to 5701761
Data columns (total 4 columns):
WELL_NAME        object
DEPT             float64
GR:1             float64
WELL_NAME_UWI    object
dtypes: float64(2), object(2)
memory usage: 174.0+ MB


In [3]:
las_data = las_data.rename(columns={'GR:1': 'CILD'})
las_data.head()

,WELL_NAME,DEPT,CILD,WELL_NAME_UWI
0,300,4.2,NaN,198
1,300,4.3,NaN,198
2,300,4.4,NaN,198
3,300,4.5,NaN,198
4,300,4.6,NaN,198


In [4]:
real_top = pd.read_csv('real_data_test\\top_df.csv', delimiter=';')
real_top = real_top.rename(columns={'WELL_NAME': 'WELL_NAME_UWI'})
del real_top['X']
del real_top['Y']
real_top.head(n=2)

,WELL_NAME_UWI,DEPTH_TOP
0,100,2548.81
1,101,2556.23


In [5]:
real_bottom = pd.read_csv('real_data_test\\bottom_df.csv', delimiter=';')
real_bottom = real_bottom.rename(columns={'WELL_NAME': 'WELL_NAME_UWI'})
del real_bottom['X']
del real_bottom['Y']
real_bottom.head(n=2)

,WELL_NAME_UWI,DEPTH_BOTTOM
0,100,2564.81
1,101,2571.71


In [6]:
# данные от организаторов
water_content_data = pd.read_csv(main.data_dir + "\\water_content.csv", delimiter=';')
water_content_data = water_content_data.rename(columns={'Скважина': main.keys_dict[main.kid_well]})
water_content_data = water_content_data.rename(columns={' Начальная обводненность ': 'WC'})
water_content_data = water_content_data.rename(columns={'Дата запуска': 'date_start'})
del water_content_data['date_start']
water_content_data = water_content_data.rename(columns={'well_name': 'WELL_NAME'})
water_content_data.head(n=5)
water_content_data['WELL_NAME'] = water_content_data['WELL_NAME'].astype('str')
water_content_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 2 columns):
WELL_NAME    181 non-null object
WC           181 non-null float64
dtypes: float64(1), object(1)
memory usage: 2.9+ KB


In [7]:
print(len(las_data['WELL_NAME'].value_counts()))
print(len(real_bottom['WELL_NAME_UWI'].value_counts()))
print(len(real_top['WELL_NAME_UWI'].value_counts()))
print(len(water_content_data['WELL_NAME'].value_counts()))

212
230
230
180


In [10]:
key_las = las_data['WELL_NAME'].value_counts().index.tolist()
key_wc = water_content_data['WELL_NAME'].value_counts().index.tolist()

In [11]:
real_data = pd.merge(water_content_data, las_data, on='WELL_NAME')
real_data = pd.merge(real_data, real_top, on='WELL_NAME_UWI')
real_data = pd.merge(real_data, real_bottom, on='WELL_NAME_UWI') 

In [12]:
real_data_na = real_data.dropna()
print(len(real_data['WELL_NAME'].value_counts()))
real_data.info()

151
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4099865 entries, 0 to 4099864
Data columns (total 7 columns):
WELL_NAME        object
WC               float64
DEPT             float64
CILD             float64
WELL_NAME_UWI    object
DEPTH_TOP        float64
DEPTH_BOTTOM     float64
dtypes: float64(5), object(2)
memory usage: 250.2+ MB


In [13]:
import lasio
import re
def load_and_convert_to_interp(dev_path, well_name):
    f = open(dev_path + well_name + '.dev', 'r')
    well_num = 0
    md = []
    abs = []
    for line in f.readlines():
        if well_num > 16:
            # list = line.split(' ')
            # print(list)
            numbers = re.findall(r'[-]?[0-9]+.[0-9]+', line)
            md.append(float(numbers[0]))
            abs.append(float(numbers[3]))
        well_num = well_num +1
    f.close()
    f_spline = interpolate.interp1d(abs, md, kind = 'slinear', bounds_error=False)
    return f_spline

In [14]:
from scipy import interpolate
dev_path = "..\\tasks\\task 6\\data\\dev\\"
well_name_list = real_data_na['WELL_NAME_UWI'].value_counts().index.tolist()
well_name = well_name_list[0]
f_spline = load_and_convert_to_interp(dev_path, well_name)
print(f_spline(0))

113.11000000000001


In [15]:
real_data_na = real_data_na.drop(real_data_na[real_data_na['WELL_NAME_UWI'] == '179'].index)

In [16]:
from scipy import interpolate
well_name_list = real_data_na['WELL_NAME_UWI'].value_counts().index.tolist()
x_values = []
y_values = []
count_val = 100
well_count = 0
for well_name in well_name_list:
    if well_count%20 == 0:
        print(well_count, ' of ', len(well_name_list))
    data_well =  real_data_na[real_data_na['WELL_NAME_UWI'] == well_name]
    f_spline = load_and_convert_to_interp(dev_path, well_name)
    bottom = f_spline(data_well['DEPTH_BOTTOM'].tolist()[0])
    top = f_spline(data_well['DEPTH_TOP'].tolist()[0])
    data_well_by_bound = data_well[(data_well['DEPT'] >= top) & (data_well['DEPT'] <= bottom)]
    x_arr = data_well_by_bound['DEPT']
    y_arr = data_well_by_bound['CILD']
    
    # print ('length of array depth', len(x_arr))
    # print(len(x_arr), len(y_arr))
    if len(x_arr) < 10:
        continue
    f_spline = interpolate.interp1d(x_arr, y_arr, kind = 'slinear')
    h_start = data_well_by_bound['DEPT'].min()
    h_end = data_well_by_bound['DEPT'].max()
    # print(h_start, h_end, top, bottom)
    h_step = (h_end - h_start)/count_val
    x_temp = []
    i = 0
    while(i < count_val):
        x_temp.append(float(f_spline(h_start + i * h_step)))
        i = i + 1
    x_values.append(x_temp)  
    y_values.append(data_well['WC'].tolist()[0])
    well_count = well_count + 1
print('end fill')

0  of  150
0  of  150
0  of  150
20  of  150
40  of  150
60  of  150
80  of  150
100  of  150
end fill


In [17]:
y_values_r = [int(10 * y_val) for y_val in y_values]

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import cross_val_score

x_train, x_hold, y_train, y_hold = train_test_split(x_values, y_values_r, test_size=0.3, random_state=17)

D:\libs\Anaconda\3.3\envs\aind\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [19]:
tree = DecisionTreeClassifier(max_depth=20, random_state=17, criterion='gini')
tree.fit(x_train, y_train)
tree_pred = tree.predict(x_hold)
accuracy_score(y_hold, tree_pred)

0.34375

In [25]:
# обучаем дерево
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score

forest = RandomForestClassifier(n_estimators=10, n_jobs=-1, random_state=17)
# print(np.mean(cross_val_score(forest, x_hold, y_hold, cv=6))) # 0.949


forest_params = { 
    'max_depth': [1, 10, 15, 20],
    'n_estimators': [1, 2, 10, 50, 100, 200, 500],
    'max_features': ['auto', 'sqrt', 'log2']
}

forest_grid = GridSearchCV(forest, forest_params, cv=6, n_jobs=-1, verbose=True)
 
forest_grid.fit(x_train, y_train)

Fitting 6 folds for each of 84 candidates, totalling 504 fits


D:\libs\Anaconda\3.3\envs\aind\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=6.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 504 out of 504 | elapsed:  1.3min finished


GridSearchCV(cv=6, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=17,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [1, 10, 15, 20], 'n_estimators': [1, 2, 10, 50, 100, 200, 500], 'max_features': ['auto', 'sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)

In [26]:
print('forest_grid.best_params_ = ', forest_grid.best_params_)
print('forest_grid.best_score_ = ', forest_grid.best_score_)
print('accuracy_score: ', accuracy_score(y_hold, forest_grid.predict(x_hold)))

forest_grid.best_params_ =  {'max_depth': 1, 'max_features': 'auto', 'n_estimators': 50}
forest_grid.best_score_ =  0.277777777778
accuracy_score:  0.25


In [30]:
import numpy as np
np.max(x_train)

23.211600000000001